In [3]:
import pandas as pd
import numpy as np

In [4]:
# Compute dist between atoms 1 & 2
# Inputs: coords_1 - atom 1 coords, coords_2 - atom 2 coords.
# Output: dist     - distance btwn atoms 1 & 2
def absolute_Distance(coords_1, coords_2):
    dist = np.sqrt((coords_2[0] - coords_1[0])**2 + (coords_2[1] - coords_1[1])**2 + (coords_2[2] - coords_1[2])**2)
    return dist

**Specify the cluster size: N**

Here, specify the size, N, of the LJ cluster you want to analyze.
Download the corresponding file, which contains the xyz coordinates of
each atom in this size-N cluster.

In [6]:
import requests

N = input("Specify cluster size: ")
N = int(N)

# N > 151 or N < 2 are inaccessible
while N > 150 or N < 3:
    N = input("Specify a cluster size less than or equal to 150 and greater than or equal to 3: ") 
    N = int(N)

# URL of the file you want to download
url = f"http://doye.chem.ox.ac.uk/jon/structures/LJ/points/{N}"

# send a GET request to the URL
response = requests.get(url)

# specify the path where the file will be saved
file_name = f"LJ_cluster_size_{N}.txt"
file_path = f"C:\\Users\\mrmuf\\OneDrive\\Desktop\\chem193\\{file_name}"

# save the file
with open(file_path, "wb") as file:
    file.write(response.content)

ModuleNotFoundError: No module named 'requests'

**Create a list of every atom's xyz coordinates**

Next, we'll clean the .txt of whitespace, and format it for use as a dataframe object - we'll store its contents in a 'raw list' of unformatted coordinates.

Then, we'll segment every 3 coordinates into lists, which each represent an atom - every element will represent one atom's x-, y-, and z-coordinates.

In [166]:
### Implementation - scraping coords into a list ###
list_of_atom_coords = [] # raw list of unsegmented atom coords
with open(file_name) as file:
    text   = file.read()
    coords = text.split()
    for coord in coords:
        list_of_atom_coords.append(coord)
    # print(list_of_atom_coords)

formatted_coords = [] # formatted list of coords ~ each atom
for coord in list_of_atom_coords:
    coords_len = len(list_of_atom_coords)
    formatted_coords = [list_of_atom_coords[i: i + 3] for i in range(0, coords_len, 3)] # split every 3 coords into a list for one atom
# print(formatted_coords)

# convert all coords to float for all N atoms (N = len(formatted_coords))
for i in range(len(formatted_coords)):
    formatted_coords[i] = list(map(float, formatted_coords[i]))

**Compute Relative Distances**

Then, we'll use absolute_Distance() to compute the relative distances between each atom in the cluster; and store these distances in a list.

In [167]:
from itertools import combinations

distances = []

# Generate all pairs of atoms & calculate relative distances
for (atom1, atom2) in combinations(formatted_coords, 2):
    dist = absolute_Distance(atom1, atom2)
    distances.append(float(dist))
    
print(distances)

[1.1224620329352735, 1.122461797373606, 1.1224619698139453]


In [168]:
# Inputs: x1 and x2 are lists that contains the x, y, and z coordinates of an atom
def lennard_Jones_PE(atom1, atom2):
    r = absolute_Distance(atom1, atom2)
    U = 4*((1/r)**12 - (1/r)**6)
    return U

In [ ]:
def update_pairwiseInt(natom, coords):
    for i in range(natom):
        # Grab the row'th atom in the position matrix
        atom1 = coords[i]
        for j in range(i, natom):
            # Check if we are calculating U_ii. If we are, leave it as 0
            if i == j:
                return 
            else:
                atom2 = coords[j]
                pIntMatrix[i][j] = lennard_Jones_PE(atom1, atom2)
                pIntMatrix[j][i] = lennard_Jones_PE(atom1, atom2)
    return 

In [169]:
def tot_Energy():
    tot_Energy = 0
    for i in range(natom):
        for j in range(i, atom):
            tot_Energy += pIntMatrix[i][j]
    return tot_Energy